In [15]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, recall_score

In [3]:
!pip install gdown --quiet
!gdown https://drive.google.com/uc?id=1T6bEwBtR4cdt8T_vGjB6LQ5MMnUMBMWB

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Downloading...
From (original): https://drive.google.com/uc?id=1T6bEwBtR4cdt8T_vGjB6LQ5MMnUMBMWB
From (redirected): https://drive.google.com/uc?id=1T6bEwBtR4cdt8T_vGjB6LQ5MMnUMBMWB&confirm=t&uuid=9946dbde-76d8-4402-81a4-7b6dd2a45417
To: /kaggle/working/rename.zip
100%|██████████████████████████████████████| 2.00G/2.00G [00:21<00:00, 93.9MB/s]


In [4]:
!unzip -q -o rename.zip -d Rice_Crop


In [42]:
main_dir = "Rice_Crop"
classes = sorted([d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))])
print(classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

['Rice___Brown_Spot', 'Rice___Healthy', 'Rice___Leaf_Blast', 'Rice___Neck_Blast']


device(type='cuda')

In [43]:

# Data augmentation and normalization for training, only normalization for testing
transform_train = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [45]:
train_dataset = datasets.ImageFolder(main_dir, transform=transform_train)

# Split the dataset into 90% training and 10% validation
train_size = int(0.9 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, _ = random_split(train_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

# Apply transform_test to the test dataset
test_dataset = datasets.ImageFolder(main_dir, transform=transform_test)
_, test_dataset = random_split(test_dataset, [train_size, test_size])
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [46]:
model = models.resnet50(pretrained=True)

# Freeze all layers except the last one
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer for our custom output size
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Linear(256, len(classes))
)

model = model.to(device)

In [ ]:
# AdamW optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.00005)

# Loss function
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
        total += labels.size(0)
    
    train_loss = running_loss / total
    train_accuracy = running_corrects.double() / total

    model.eval()
    running_corrects = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total += labels.size(0)
            running_corrects += torch.sum(preds == labels.data)
    
    test_accuracy = running_corrects.double() / total

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

torch.save(model.state_dict(), "Rice_model.pth")

Epoch 1/50
Train Loss: 1.3259, Train Accuracy: 0.4417
Test Accuracy: 0.3725
Epoch 2/50
Train Loss: 1.2103, Train Accuracy: 0.5463
Test Accuracy: 0.5760
Epoch 3/50
Train Loss: 1.1110, Train Accuracy: 0.6120
Test Accuracy: 0.5980
Epoch 4/50
Train Loss: 1.0275, Train Accuracy: 0.6188
Test Accuracy: 0.5980
Epoch 5/50
Train Loss: 0.9600, Train Accuracy: 0.6223
Test Accuracy: 0.6225
Epoch 6/50
Train Loss: 0.9121, Train Accuracy: 0.6580
Test Accuracy: 0.6520
Epoch 7/50
Train Loss: 0.8713, Train Accuracy: 0.6520
Test Accuracy: 0.6642


In [28]:
!ls

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Rice_Crop  Rice_model.pth  rename.zip


In [33]:
%cd Rice_Crop/Rice___Brown_Spot/IMG_20190419_095712.jpg
!ls

/kaggle/working/Rice_Crop
Rice___Brown_Spot  Rice___Healthy  Rice___Leaf_Blast  Rice___Neck_Blast


In [35]:
%cd ../..
!ls

/kaggle/working
Rice_Crop  Rice_model.pth  rename.zip


In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Path to the image you want to test (replace with your image file path)
image_path = r"Rice_Crop/Rice___Brown_Spot/IMG_20190419_095712.jpg"  # Specify the image file path

# Load the saved model
model = models.resnet50(pretrained=False)  # Don't load pretrained weights, we have our model
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Linear(256, len(classes))
)

model.load_state_dict(torch.load("Rice_model.pth"))
model = model.to(device)
model.eval()

In [37]:
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the image and apply transformations
image = Image.open(image_path)
image = transform_test(image).unsqueeze(0).to(device) 

In [38]:
# Get the model's prediction
with torch.no_grad():
    outputs = model(image)
    _, predicted_class = torch.max(outputs, 1)

# Map the predicted class index to the actual class label
predicted_label = classes[predicted_class.item()]
print(f"Predicted class: {predicted_label}")

Predicted class: Rice___Healthy


In [40]:
predicted_class

tensor([1], device='cuda:0')